**Note**:
First of all you must have training.csv, for which you can see my previous notebook, where i have written code on how to make training.csv.

In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/training-df/training_df.csv')
train_df.head()

In [ ]:
#Helper Function to decode RLE
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

In [ ]:
#Create Directories to seperately save Images and Masks.
import os
if not os.path.exists('./Images/'):
    os.mkdir('./Images/')
if not os.path.exists('./Masks/'):
    os.mkdir('./Masks/')

images_dir = './Images/'
masks_dir = './Masks/'

In [ ]:
# this function will take training dataframe and will create and store masks
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import math

def create_and_store_masks(df):
    colors = [(255,0,0), (0,255,0), (0,0,255)] #red green blue respectively
    classes = ['large_bowel', 'small_bowel', 'stomach']
    x=0 #counter for images and masks
    for i in range(0, len(df)):
        #get necessary items from df   
                
        path = df.iloc[i]['path']
        img = plt.imread(path)
        pimg = cv.imread(path)
        lb_class_seg = df.iloc[i]['large_bowel']
        sb_class_seg = df.iloc[i]['small_bowel']
        st_class_seg = df.iloc[i]['stomach']
        
        #define black masks
        lb_mask = sb_mask = st_mask = np.zeros(pimg.shape, dtype='float32') 
        
        #get real masks of each class
        
        if not math.isnan(float(str(lb_class_seg).replace(" ",""))):
            lb_mask = rle_decode(lb_class_seg, pimg.shape, color=colors[0])
        if not math.isnan(float(str(sb_class_seg).replace(" ", ""))):
            sb_mask = rle_decode(sb_class_seg, pimg.shape, color=colors[1])
        if not math.isnan(float(str(st_class_seg).replace(" ",""))):
            st_mask = rle_decode(st_class_seg, pimg.shape, color=colors[2])
            
        #merge masks
        mask = cv.bitwise_or(lb_mask, sb_mask)
        mask = cv.bitwise_or(mask, st_mask)
        
        plt.imsave(images_dir+str(x)+'.png', img)
        plt.imsave(masks_dir+str(x)+'.png', mask/255.0)
        
        print(x)
        
        x += 1

In [ ]:
#store images and masks into respective directories 
create_and_store_masks(train_df) #it will take some time cz there are too many images.....

In [ ]:
#images and masks
images = os.listdir(images_dir)
masks = os.listdir(masks_dir)
len(images), len(masks)

In [ ]:
#Visualize Saved Images and masks
import random
rand_index = random.randint(0, len(images))
img = plt.imread(images_dir+images[rand_index])
mask = plt.imread(masks_dir + images[rand_index])
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title(f"Image Shape= {img.shape}")
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(mask)
plt.title(f"Mask Shape = {mask.shape}")
plt.axis('off')
print(f"Image max: {img.max()} \tImage min: {img.min()}\nMask max: {mask.max()} \tMask min: {mask.min()} \tMask unique values: {np.unique(mask)}")

If you got any help from this kindly support by **Upvoting**.

Thanks.